# 三种建模方式

## Sequential model

### 第一种Sequential model

In [1]:
from tensorflow.keras import layers
import tensorflow as tf
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))#第一层
model.add(layers.Dense(64, activation='relu'))#第二层
model.add(layers.Dense(10))#第三层
# 。。。。。。。

### 第二种Sequential model

In [2]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu', input_shape=(32,)),#第一层
layers.Dense(64, activation='relu'),#第二层
layers.Dense(10)#第三层
    #。。。。。
])

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [4]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 38.8803 - accuracy: 0.0810
Epoch 2/10
1000/1000 [==============================] - 1s 694us/sample - loss: 164.6640 - accuracy: 0.1040
Epoch 3/10
1000/1000 [==============================] - 1s 513us/sample - loss: 693.8720 - accuracy: 0.1150
Epoch 4/10
1000/1000 [==============================] - 1s 597us/sample - loss: 2223.4278 - accuracy: 0.1080
Epoch 5/10
1000/1000 [==============================] - 0s 365us/sample - loss: 4529.5607 - accuracy: 0.0900
Epoch 6/10
1000/1000 [==============================] - 1s 643us/sample - loss: 7086.6511 - accuracy: 0.0990
Epoch 7/10
1000/1000 [==============================] - 0s 324us/sample - loss: 9427.3324 - accuracy: 0.0860
Epoch 8/10
1000/1000 [==============================] - 1s 514us/sample - loss: 14321.9221 - accuracy: 0.1160
Epoch 9/10
1000/1000 [==============================] - 0s 486us/sample - loss: 20877.1866 - accuracy: 0.0850
E

tf.keras.Model.fit (用于模型训练) 

- epochs：训练分为几个时期。每一个epoch是对整个输入数据的一次迭代（此操作以较小的批次完成）。
- batch_size：当传递NumPy数据时，模型将数据切成较小的批次，并在训练期间对这些批次进行迭代。该整数指定每个批次的大小。请注意，如果不能将样本总数除以批次大小，则最后一批可能会更小。
- validation_data：在模型训练时，监控在某些验证数据上监视其性能。传递此参数（输入和标签的元组）可以使模型在每个时期结束时以推断模式显示所传递数据的损失和度量。

##  Functional model

函数式模型是一种创建模型的方法，该模型比tf.keras.Sequential更灵活。函数式模型可以处理具有非线性拓扑的模型，具有共享层的模型以及具有多个输入或输出的模型等等

深度学习模型通常是层的有向无环图（DAG）的主要思想。因此，函数式模型是一种构建层图的方法。



举个例子：
"""
(input: 32-dimensional vectors) \
       ↧
[Dense (64 units, relu activation)] \
       ↧
[Dense (64 units, relu activation)] \
       ↧
[Dense (10 units, softmax activation)] \
       ↧
(output: logits of a probability distribution over 10 classes)
"""

In [46]:
inputs = tf.keras.Input(shape=(32,))  
# inputs = tf.keras.Input(shape=(32,))  
x = layers.Dense(64, activation='relu')(inputs) #第一层
x = layers.Dense(64, activation='relu')(x) #第二层
predictions = layers.Dense(10)(x) #第三层

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
model.fit(data, labels, batch_size=32, epochs=5)

In [9]:
inputs1 = tf.keras.Input(shape=(32,))  #输入1
inputs2 = tf.keras.Input(shape=(32,))  #输入2
x1 = layers.Dense(64, activation='relu')(inputs1) #第一层
x2 = layers.Dense(64, activation='relu')(inputs2) #第一层
x = tf.concat([x1,x2],axis=-1)
x = layers.Dense(64, activation='relu')(x) #第二层
predictions = layers.Dense(10)(x) #第三层

In [10]:
model = tf.keras.Model(inputs=[inputs1,inputs2], outputs=predictions)


model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np
data1 = np.random.random((1000, 32))
data2 = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
model.fit((data1,data2), labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 18.0165 - accuracy: 0.0920
Epoch 2/5
1000/1000 [==============================] - 0s 261us/sample - loss: 45.0426 - accuracy: 0.1050
Epoch 3/5
1000/1000 [==============================] - 0s 180us/sample - loss: 84.2061 - accuracy: 0.1150
Epoch 4/5
1000/1000 [==============================] - 0s 160us/sample - loss: 123.1821 - accuracy: 0.1000
Epoch 5/5
1000/1000 [==============================] - 0s 488us/sample - loss: 150.3576 - accuracy: 0.1060


## 模型子类化 (重要！！！)

通过子类化tf.keras.Model和定义自己的前向传播模型来构建完全可定制的模型。

和eager execution模式相辅相成。


In [11]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = layers.Dense(32, activation='relu') #
        self.dense_2 = layers.Dense(num_classes)

    def call(self, inputs):
        #定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return x

In [12]:
model = MyModel(num_classes=10)

model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 853us/sample - loss: 12.9870 - accuracy: 0.1170
Epoch 2/5
1000/1000 [==============================] - 0s 328us/sample - loss: 14.9116 - accuracy: 0.1120
Epoch 3/5
1000/1000 [==============================] - 0s 269us/sample - loss: 17.3875 - accuracy: 0.1080
Epoch 4/5
1000/1000 [==============================] - 0s 123us/sample - loss: 19.5177 - accuracy: 0.1110
Epoch 5/5
1000/1000 [==============================] - 0s 274us/sample - loss: 21.4926 - accuracy: 0.1270
